In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Load documents
loader = PyPDFDirectoryLoader("documents/Verträge/")
documents = loader.load()

print(f"Loaded {len(documents)} documents.")

Loaded 6 documents.


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

def custom_length_function(text):
    return len(text) + text.count('§') * 1000

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", ", ", " ", ""],
    chunk_size=2000,
    chunk_overlap=0,
    length_function=custom_length_function,
    is_separator_regex=False,
)

def split_by_section(text):
    sections = re.split(r'(?=\n\s*§)', text)
    return [section.strip() for section in sections if section.strip()]

# Split documents into chunks
split_docs = []
for doc in documents:
    sections = split_by_section(doc.page_content)
    
    for section in sections:
        if custom_length_function(section) <= 2000:
            split_docs.append({"content": section, "metadata": doc.metadata})
        else:
            splits = text_splitter.split_text(section)
            split_docs.extend([{"content": split, "metadata": doc.metadata} for split in splits])

print(f"Created {len(split_docs)} chunks.")
print(split_docs)

Created 17 chunks.
[{'content': 'Standardliefervertrag  \n \nzwischen  \n \nder Firma Textilius GmbH, vertreten durch den Geschäftsführer Max Mustermann,  \nIndustriestraße 1, 12345 Musterstadt  \n- nachfolgend "Lieferant" genannt - \n \nund  \n \nder Firma Schneiderei Schmidt KG, vertreten durch die Geschäftsführerin Anna \nSchmidt,  \nIndustriepark 3, 54321 Musterdorf  \n- nachfolgend "Kunde" genannt -', 'metadata': {'source': 'documents\\Verträge\\Standardliefervertrag.pdf', 'page': 0}}, {'content': '§ 1 Vertragsgegenstand  \n \n1.1 Gegenstand dieses Vertrages ist die Lieferung von Textilmaterialien durch den \nLieferanten an den Kunden gemäß den nachfolgenden Bestimmungen.  \n \n1.2 Die zu liefernden Produkte umfassen:  \n   a) Baumwollstoff, 100m Rolle  \n   b) Polyesterfaden, 50m Spule  \n   c) Reißverschlüsse, 25cm', 'metadata': {'source': 'documents\\Verträge\\Standardliefervertrag.pdf', 'page': 0}}, {'content': '§ 2 Vertragslaufzeit  \n \n2.1 Dieser Vertrag beginnt am 01.01.20

In [3]:
# Output the chunks from split_docs for review
print(f"Total chunks created: {len(split_docs)}")
print("\nReviewing the chunks:\n")

for i, doc in enumerate(split_docs):
    print(f"Chunk {i+1}:")
    print(doc['content'])
    print("\n" + "="*50 + "\n")

Total chunks created: 17

Reviewing the chunks:

Chunk 1:
Standardliefervertrag  
 
zwischen  
 
der Firma Textilius GmbH, vertreten durch den Geschäftsführer Max Mustermann,  
Industriestraße 1, 12345 Musterstadt  
- nachfolgend "Lieferant" genannt - 
 
und  
 
der Firma Schneiderei Schmidt KG, vertreten durch die Geschäftsführerin Anna 
Schmidt,  
Industriepark 3, 54321 Musterdorf  
- nachfolgend "Kunde" genannt -


Chunk 2:
§ 1 Vertragsgegenstand  
 
1.1 Gegenstand dieses Vertrages ist die Lieferung von Textilmaterialien durch den 
Lieferanten an den Kunden gemäß den nachfolgenden Bestimmungen.  
 
1.2 Die zu liefernden Produkte umfassen:  
   a) Baumwollstoff, 100m Rolle  
   b) Polyesterfaden, 50m Spule  
   c) Reißverschlüsse, 25cm


Chunk 3:
§ 2 Vertragslaufzeit  
 
2.1 Dieser Vertrag beginnt am 01.01.2024 und endet am 31.12.2024.


Chunk 4:
2.2 Der Vertrag verlängert sich automatisch um jeweils ein weiteres Jahr, sofern er nicht 
von einer der Parteien mit einer Frist von drei 

In [4]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def get_section_heading(current_chunk, previous_chunks, next_chunks):
    context = "\n\n".join(previous_chunks + [current_chunk] + next_chunks)
    
    system_prompt = """You are an assistant that analyzes legal document chunks. 
                        Your task is to determine if the current chunk needs a section heading. 
                        If it does, provide ONLY the exact heading as it appears in the context. 
                        If not, respond with ONLY 'No heading needed'. 
                        Do not include any explanatory text in your response."""
    
    user_input = f"""Analyze this document chunk and its context. 
                    Determine if the current chunk (marked with '===CURRENT CHUNK===') needs a section heading. 
                    If it does, provide ONLY the exact heading as it appears in the context. 
                    The heading should start with '§' followed by a number, or be an exact match to a heading in the context. 
                    If no heading is needed, respond with ONLY 'No heading needed'.Context:\n{context}\n\n===CURRENT CHUNK===\n{current_chunk}"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )
    return response.choices[0].message.content.strip()

print("GPT-4 interaction function defined.")

GPT-4 interaction function defined.


In [5]:
def process_chunks(chunks):
    processed_chunks = []
    current_heading = ""
    WINDOW_SIZE = 3  # Number of chunks to consider for context
    
    for i, chunk in enumerate(chunks):
        content = chunk['content']
        
        # Determine context chunks
        start_idx = max(0, i - WINDOW_SIZE)
        end_idx = min(len(chunks), i + WINDOW_SIZE + 1)
        previous_chunks = [c['content'] for c in chunks[start_idx:i]]
        next_chunks = [c['content'] for c in chunks[i+1:end_idx]]
        
        # Process chunk
        if content.strip().startswith('§'):
            current_heading = content.split('\n')[0]
            processed_chunks.append(chunk)
        else:
            gpt_response = get_section_heading(content, previous_chunks, next_chunks)
            
            if gpt_response != "No heading needed":
                new_content = f"{gpt_response}\n{content}"
                processed_chunks.append({"content": new_content, "metadata": chunk['metadata']})
                current_heading = gpt_response
            elif current_heading:
                new_content = f"{current_heading}\n{content}"
                processed_chunks.append({"content": new_content, "metadata": chunk['metadata']})
            else:
                processed_chunks.append(chunk)
    
    return processed_chunks

# Process the chunks
processed_docs = process_chunks(split_docs)

print(f"Processed {len(processed_docs)} chunks with GPT-4.")

Processed 17 chunks with GPT-4.


In [6]:
# Example: Outputting the processed chunks for review
for i, doc in enumerate(processed_docs):  # Display first 5 chunks as an example
    print(f"Chunk {i+1}:")
    print(doc)
    print("\n" + "="*50 + "\n")

Chunk 1:
{'content': 'Standardliefervertrag  \n \nzwischen  \n \nder Firma Textilius GmbH, vertreten durch den Geschäftsführer Max Mustermann,  \nIndustriestraße 1, 12345 Musterstadt  \n- nachfolgend "Lieferant" genannt - \n \nund  \n \nder Firma Schneiderei Schmidt KG, vertreten durch die Geschäftsführerin Anna \nSchmidt,  \nIndustriepark 3, 54321 Musterdorf  \n- nachfolgend "Kunde" genannt -', 'metadata': {'source': 'documents\\Verträge\\Standardliefervertrag.pdf', 'page': 0}}


Chunk 2:
{'content': '§ 1 Vertragsgegenstand  \n \n1.1 Gegenstand dieses Vertrages ist die Lieferung von Textilmaterialien durch den \nLieferanten an den Kunden gemäß den nachfolgenden Bestimmungen.  \n \n1.2 Die zu liefernden Produkte umfassen:  \n   a) Baumwollstoff, 100m Rolle  \n   b) Polyesterfaden, 50m Spule  \n   c) Reißverschlüsse, 25cm', 'metadata': {'source': 'documents\\Verträge\\Standardliefervertrag.pdf', 'page': 0}}


Chunk 3:
{'content': '§ 2 Vertragslaufzeit  \n \n2.1 Dieser Vertrag beginnt am

## HTTP Client Chroma

In [8]:
import chromadb
import openai
from dotenv import load_dotenv
import os
import chromadb.utils.embedding_functions as embedding_functions

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")

# Use OpenAIEmbeddingFunction from Chroma utils
embedding_function = embedding_functions.OpenAIEmbeddingFunction(
    api_key=openai.api_key,
    model_name="text-embedding-3-large"  # You can choose another model like "text-embedding-ada-002"
)

# Set up the Chroma HTTP client to connect to your AWS instance
chroma_client = chromadb.HttpClient(
    host='16.171.0.22',  # Replace with your AWS instance's public IP
    port=8000
)


In [9]:
# Assuming processed_docs is a list of chunks (processed document parts)
# Add the processed chunks into ChromaDB

collection = chroma_client.get_or_create_collection(
    name="bachelor-vertrag",
    embedding_function=embedding_function
)

# Example processed docs
# processed_docs = [{"content": "Standardliefervertrag ...", "metadata": {"source": "path_to_pdf", "page": 0}}]

# Extract the document content and metadata for insertion into the database
documents = [doc['content'] for doc in processed_docs]
metadatas = [doc['metadata'] for doc in processed_docs]
ids = [f"doc_{i+1}" for i in range(len(processed_docs))]  # Unique IDs for each document

# Add to the collection
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)


In [10]:
# Test querying the collection
result = collection.query(
    query_texts=["Standardliefervertrag"],  # Example query
    n_results=5,
    include=["documents", "metadatas"]
)

# Access the 'documents' key from the result dictionary
documents = result.get("documents")
metadatas = result.get("metadatas")

# Print the documents and metadata
print("Documents:", documents)
print("Metadata:", metadatas)


Documents: [['Standardliefervertrag  \n \nzwischen  \n \nder Firma Textilius GmbH, vertreten durch den Geschäftsführer Max Mustermann,  \nIndustriestraße 1, 12345 Musterstadt  \n- nachfolgend "Lieferant" genannt - \n \nund  \n \nder Firma Schneiderei Schmidt KG, vertreten durch die Geschäftsführerin Anna \nSchmidt,  \nIndustriepark 3, 54321 Musterdorf  \n- nachfolgend "Kunde" genannt -', '§ 12 Schlussbestimmungen  \n \n12.1 Änderungen und Ergänzungen dieses Vertrages bedürfen der Schriftform. Dies gilt \nauch für die Aufhebung des Schriftformerfordernisses.  \n \n12.2 Sollten einzelne Bestimmungen dieses Vertrages unwirksam oder undurchführbar \nsein oder werden, so berührt dies die Wirksamkeit der übrigen Bestimmungen nicht.  \n \n12.3 Es gilt ausschließlich das Recht der Bundesrepublik Deutschland unter Ausschluss \ndes UN-Kaufrechts.  \n \n12.4 Erfüllungsort und Gerichtsstand für alle Streitigkeiten aus diesem Vertrag ist \nMusterstadt, sofern der Kunde Kaufmann, juristische Person 

## Old way

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
from langchain.schema import Document


# Convert processed_docs to the format expected by Chroma
chunk_objects = [
    Document(
        page_content=doc['content'],
        metadata=doc['metadata']
    )
    for doc in processed_docs
]


from langchain.vectorstores import Chroma

vectordb = Chroma.from_documents(
    documents=chunk_objects,
    embedding=embeddings,
    persist_directory="vectordb/vertrag"
)

print(f"Created vectorstore with {len(chunk_objects)} documents.")

In [ ]:
results = vectordb.similarity_search(
    "Welche Bedingungen gelten für die Preisänderungen und wie wirken sich diese auf die Vertragslaufzeit aus?",
    k=5,
    #filter={"source": "documents\\Verträge\\Lieferzeitvereinbarung.pdf"},
)
for res in results:
    print(f"{res.page_content} \n\n [{res.metadata}]")

In [16]:
results = vectordb.similarity_search(
    "Welche Lieferzeit ist vereinbart?",
    k=1,
    filter={"source": "Standardliefervertrag"},
)

# Ausgabe der Suchergebnisse
for res in results:
    print(f"{res.page_content} \n\n [{res.metadata}]")


In [ ]:

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Beantworte diese FRAGE nur unter Verwendung des bereitgestellten KONTEXT."},
        {"role": "user", "content": prompt}
    ],
    temperature=0
)

answer = response['choices'][0]['message']['content']
